In [1]:
!pip install -q peft==0.8.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.8 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments

model_name = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
import re

def clean_text(text):
    # replace all the special characters with space
    text=text.replace('+',' plus ')
    text=text.replace('-',' minus ')
    text=text.replace('*',' times ')
    text=text.replace('^',' power ')
    text=text.replace('/',' divide ')
    text=text.replace('<',' le ')
    text=text.replace('>',' ge ')
    text=text.replace('≤',' leq ')
    text=text.replace('<=',' leq ')
    text=text.replace('≥',' geq ')
    text=text.replace('>=',' geq ')
    text=text.replace('=',' equal ')
    text=text.replace('#',' numerical ')

    text = re.sub(r'[_\'\"]', '', text)

    text = re.sub(r'\W', ' ', text)
    return text

In [4]:
df = pd.read_json('test.json').replace(np.nan, 0)
texts=df["statement"].apply(str.lower).apply(clean_text)
texts

,statement
0,it is never too late to play the fancy binary ...
1,instructors of some informatics school make st...
2,bigdata inc is a corporation that has n data ...
3,dima is a beginner programmer during his work...
4,oleg writes down the history of the days he li...
...,...
702,igor found out discounts in a shop and decided...
703,polycarp is crazy about round numbers he espe...
704,in berland each high school student is charact...
705,boris really likes numbers and even owns a sma...


In [5]:
# Get all columns that start with 'tags.'
tag_columns = [col for col in df.columns if col.startswith('tags.')]
print('tag_columns:', tag_columns)
print('tag_columns count:', len(tag_columns))
# Get the dataframe data in those columns

tag_columns: ['tags.brute force', 'tags.binary search', 'tags.greedy', 'tags.sortings', 'tags.dfs and similar', 'tags.graphs', 'tags.constructive algorithms', 'tags.math', 'tags.implementation', 'tags.data structures', 'tags.dp', 'tags.combinatorics', 'tags.matrices', 'tags.strings', 'tags.number theory', 'tags.ternary search', 'tags.two pointers', 'tags.dsu', 'tags.bitmasks', 'tags.shortest paths', 'tags.games', 'tags.trees', 'tags.probabilities', 'tags.geometry', 'tags.string suffix structures', 'tags.*special', 'tags.interactive', 'tags.flows', 'tags.fft', 'tags.chinese remainder theorem', 'tags.divide and conquer', 'tags.meet-in-the-middle', 'tags.hashing', 'tags.graph matchings', 'tags.2-sat']
tag_columns count: 35


In [6]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(df['statement'].values.tolist(), df[tag_columns].values.tolist(), test_size=.2)

In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
class CFDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CFDataset(train_encodings, train_labels)
val_dataset = CFDataset(val_encodings, val_labels)

In [9]:
len(train_dataset)

565

In [10]:
# Cargar el modelo preentrenado y el tokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(tag_columns),problem_type = "multi_label_classification"
)
model.to('cuda')

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-5): 6 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=T

In [16]:
import peft
from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=2,
    lora_alpha=16,
    target_modules=['query_proj', 'value_proj'],
)

peft_model = get_peft_model(model, lora_config)
peft_model.to('cuda')

PeftModel(
  (base_model): LoraModel(
    (model): DebertaV2ForSequenceClassification(
      (deberta): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-5): 6 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=2, bias=False)
                    )
                    (lora_B): ModuleDict(
    

In [17]:
training_args = TrainingArguments(
    output_dir='./deberta-finetunned',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=1000,
    save_total_limit=1,                    # Mantener solo 1 versión del modelo guardado
    logging_steps=50,                      # Intervalos de logging menos frecuentes
    logging_dir="./logs",                  # Carpeta para los logs
    fp16=False,                            # Puedes probar a activarlo si tu GPU soporta fp16
    evaluation_strategy="no",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Step,Training Loss
50,0.716400
100,0.711600
150,0.710900
200,0.693800
250,0.661000
300,0.620900
350,0.602500
400,0.587500
450,0.586100
500,0.578900


TrainOutput(global_step=565, training_loss=0.638913390910731, metrics={'train_runtime': 439.9351, 'train_samples_per_second': 1.284, 'train_steps_per_second': 1.284, 'total_flos': 373309923037500.0, 'train_loss': 0.638913390910731, 'epoch': 1.0})

In [19]:
trainer.evaluate()

{'eval_runtime': 13.2337,
 'eval_samples_per_second': 10.73,
 'eval_steps_per_second': 1.36,
 'epoch': 1.0}

In [24]:
new_texts = ["You are given circular array a0 a1 an 1 There are two types of operations with it Assume segments to be circular so if n 5 and lf 3 rg 1 it means the index sequence 3 4 0 1 Write program to process given sequence of operations The first line contains integer n 1 n 200000 The next line contains initial state of the array a0 a1 an 1 106 ai 106 ai are integer The third line contains integer m 0 m 200000 m the number of operartons Next m lines contain one operation each If line contains two integer lf rg 0 lf rg n 1 it means rmq operation it contains three integers lf rg v 0 lf rg n 1 106 v 106 inc operation For each rmq operation write result for it Please do not use lld specificator to read or write 64bit integers in C It is preffered to use cout also you may use I64d",
             "You are given a n m field consisting only of periods and asterisks Your task is to count all right triangles with two sides parallel to the square sides whose vertices are in the centers of cells A right triangle is a triangle in which one angle is a right angle that is a 90 degree angle The first line contains two positive integer numbers n and m 1 n m 1000 The following n lines consist of m characters each describing the field Only and are allowed Output a single number total number of square triangles in the field Please do not use lld specificator to read or write 64bit integers in C It is preffered to use cout also you may use I64d"]
inputs = tokenizer(new_texts, return_tensors='pt', padding=True, truncation=True).to('cuda')

with torch.no_grad():
    outputs = model(**inputs)

    predictions = torch.sigmoid(outputs.logits)
    # predictions = torch.argmax(outputs.logits, dim=-1)
for prediction in predictions:
  my_preds = []
  for pred, tag in zip(prediction, tag_columns):
      my_preds.append((pred, tag))
  for pred, tag in sorted(my_preds, key=lambda x: x[0], reverse=True)[:5]:
      print(f"{tag}: {pred}")
  print()


tags.string suffix structures: 0.5096468925476074
tags.implementation: 0.5082830786705017
tags.hashing: 0.4984917938709259
tags.number theory: 0.4918721616268158
tags.brute force: 0.48295313119888306

tags.string suffix structures: 0.5134623646736145
tags.implementation: 0.50605708360672
tags.hashing: 0.5016369819641113
tags.number theory: 0.48816585540771484
tags.brute force: 0.4852346181869507

